# Entraînement Morningstar sur Google Colab

In [ ]:
# Configuration de base
!pip install -q tensorflow-gpu pyarrow wandb
import tensorflow as tf
import wandb
from google.colab import drive

# Vérification GPU
print("GPU disponible:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Téléchargement des données
!git clone https://github.com/votre_repo/Morningstar.git
%cd Morningstar

# Ou depuis Hugging Face Datasets
# from datasets import load_dataset
# dataset = load_dataset("votre_user/Morningstar-Dataset")

In [ ]:
# Configuration W&B
!wandb login a1478933771f0389426436c0de1c39585a5a452c
wandb.init(project="morningstar", entity="cabrelkaka-morningstar")

In [ ]:
# Chargement du modèle
from model.architecture.morningstar_model import MorningstarTradingModel
from configs.morningstar_config import MorningstarConfig

config = MorningstarConfig()
model = MorningstarTradingModel(input_shape=(60, 19), num_classes=3)
model.compile_model(learning_rate=0.001)

In [ ]:
# Callbacks avancés
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=15),
    wandb.keras.WandbCallback()
]

In [ ]:
# Entraînement avec stratégie Mirrored pour multi-GPU
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    history = model.model.fit(
        train_data,
        epochs=100,
        validation_data=val_data,
        callbacks=callbacks
    )

In [ ]:
# Sauvegarde sur Drive et Hugging Face
drive.mount('/content/drive')
!cp best_model.h5 '/content/drive/MyDrive/Morningstar/'

# Upload vers Hugging Face
!pip install -U huggingface_hub
!huggingface-cli login
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="best_model.h5",
    path_in_repo="model.h5",
    repo_id="Cabrel10/morningstar",
    repo_type="model"
)